In [1]:
#import webbrowser
import time
import os
import shutil
from csv import reader
import numpy as np
import glob
from csv import reader
import csv
from operator import itemgetter
import datetime

In [2]:
# def get_link(number):
#     url_start = "https://data.cusp.nyu.edu/filebrowser/download=/gws/projects/project-taxi_capstone_2016/share/breadcrumb_nad83_nearest3_150ft/part-"
#     result = str(number)
#     result = url_start + "".join((5 - len(result))*['0'])+result + ".gz"
#     return result

# def download_(number):
#     link = get_link(number)
#     webbrowser.open(link, new=0)
#     while not os.path.isfile('YellowGreen/'+ link.split('/')[-1]):
#         time.sleep(7)
#     print "Loaded file number " + str(number)

In [3]:
#m = map(download_, range(674, 2132))

In [4]:
# January
rdd = sc.textFile('YellowGreen/part*.gz')
oneday = rdd.filter(lambda x: x.split(',')[1].startswith('1504')).map(lambda x: x.split(':')[0])

In [5]:
#oneday.take(1)

In [6]:
oneday.saveAsTextFile('output1')

In [7]:
#rdd = sc.textFile('output0/part*')

In [8]:
#first = rdd.take(1)
#first[0]

In [9]:
#len(first[0]), len(first[0].split(':')[0]), first[0].split(':')[0]

In [10]:
#onestreet = rdd.map(lambda x: x.split(':')[0])

In [11]:
#onestreet.saveAsTextFile('output1')

In [12]:
#rdd = sc.textFile('output1/part*')

In [13]:
#rdd.take(1)[0].split(',')[0]

In [14]:
#taxi_ids = rdd.map(lambda x: x.split(',')[0]).distinct()

In [15]:
#taxi_ids.saveAsTextFile('taxi_id')

In [16]:
# # Merging
# with open('taxies.csv', 'wb') as outpt:
#     for name in glob.glob('taxi_id/p*'):
#         with open(name, 'rb') as inpt:
#             shutil.copyfileobj(inpt, outpt)

In [17]:
# # Collecting unique taxi ids
# filename = 'taxies.csv'
# with open(filename, 'rb') as csvfile:
#     reader = csv.reader(csvfile, delimiter=',')
#     taxies = map(lambda id_: id_[0], reader)

In [18]:
# Creating original and 1 time step lagged collections with indexing
rdd = sc.textFile('output1/part*')
rm1 = rdd.sortBy(lambda x: (x.split(',')[0], x.split(',')[1])).\
            map(lambda x: (x.split(',')[0], x.split(',', 1)[-1])).\
            zipWithIndex().map(itemgetter(1, 0))

rm2 = rdd.sortBy(lambda x: (x.split(',')[0], x.split(',')[1])).zipWithIndex().\
            filter(lambda (row, index): index > 0).keys().\
            map(lambda x: (x.split(',')[0], x.split(',', 1)[-1])).\
            zipWithIndex().map(itemgetter(1, 0))

In [19]:
#rm1.take(3)

In [20]:
#rm2.take(3)

In [21]:
# Joining and pairing consecutive records for the same taxi ids for pickup extraction
rm3 = rm1.join(rm2).sortBy(lambda x: x[0]).\
            filter(lambda x: (x[1][0][0]==x[1][1][0]) and \
                             (int(x[1][0][1].split(',')[-2])<int(x[1][1][1].split(',')[-2]))).\
            map(lambda x: x[1][1])

In [22]:
rm3.saveAsTextFile('output2')

In [23]:
# Number of pickups
rdd = sc.textFile('output2/part*')
#rdd.count()

In [24]:
lion = sc.textFile('lion_census_combined.csv')
lion = lion.zipWithIndex().filter(lambda (row, index): index > 0).keys()
#lion.count()

In [25]:
#lion.take(5)

In [26]:
def prepare_pickups(record):
    record = record.split(',')
    return (record[-1][:-2], record[1][3:])

def prepare_lion(record):
    record = record.split(',')
    return (record[0], record[1] + ',' + record[2])

In [27]:
rm1 = rdd.map(lambda x: prepare_pickups(x))
rm2 = lion.map(lambda x: prepare_lion(x))

In [28]:
#rm1.take(5), rm2.take(5)

In [29]:
rm3 = rm1.join(rm2)
rm3 = rm3.map(lambda x: x[1])
rm3.saveAsTextFile('output3')

In [30]:
def get_minutes(record):
    hourminute = record.split(',')[0][9:-3]
    hours = int(hourminute[:2])
    minutes = int(hourminute[2:]) + hours*60
    return minutes

#sunday = 6
def is_sunday(record):
    date = record.split(',')[0][3:-7]
    year = int('20'+date[:2])
    month = int(date[2:4])
    day = int(date[4:])
    return datetime.datetime(year, month, day).weekday() == 6

def time_to_minutes(time_):
    hours = int(time_[:2])
    minutes = int(time_[2:])
    return hours*60+minutes

In [31]:
rdd = sc.textFile('output3/part*').cache()
#rdd.count()

In [32]:
sunday = rdd.filter(lambda x: is_sunday(x))
weekday = rdd.filter(lambda x: not is_sunday(x))

In [33]:
#sunday.count(), weekday.count()

In [34]:
#sunday.take(3), weekday.take(3)

In [35]:
# Weekdays
_1200amto0600am = [time_to_minutes('0000'), time_to_minutes('0600')]
_0600amto1000am = [time_to_minutes('0600'), time_to_minutes('1000')]
_1000amto0330pm = [time_to_minutes('1000'), time_to_minutes('1530')]
_0330pmto0430pm = [time_to_minutes('1530'), time_to_minutes('1630')]
_0430pmto0600pm = [time_to_minutes('1630'), time_to_minutes('1800')]
_0600pmto0900pm = [time_to_minutes('1800'), time_to_minutes('2100')]
_0900pmto1200am = [time_to_minutes('2100'), time_to_minutes('2400')]

# Sundays
_1200amto0200am = [time_to_minutes('0000'), time_to_minutes('0200')]
_0200amto0200pm = [time_to_minutes('0200'), time_to_minutes('1400')]
_0200pmto0400pm = [time_to_minutes('1400'), time_to_minutes('1600')]
_0400pmto1200am = [time_to_minutes('1600'), time_to_minutes('2400')]

In [36]:
#_1200amto0600am, _0600amto1000am, _1000amto0330pm, _0330pmto0430pm, _0430pmto0600pm, _0600pmto0900pm, _0900pmto1200am

#_1200amto0200am, _0200amto0200pm, _0200pmto0400pm, _0400pmto1200am

In [37]:
weekday_1200amto0600am = weekday.\
                        filter(lambda x: get_minutes(x)>=_1200amto0600am[0] and get_minutes(x)<_1200amto0600am[1])
weekday_0600amto1000am = weekday.\
                        filter(lambda x: get_minutes(x)>=_0600amto1000am[0] and get_minutes(x)<_0600amto1000am[1])
weekday_1000amto0330pm = weekday.\
                        filter(lambda x: get_minutes(x)>=_1000amto0330pm[0] and get_minutes(x)<_1000amto0330pm[1])
weekday_0330pmto0430pm = weekday.\
                        filter(lambda x: get_minutes(x)>=_0330pmto0430pm[0] and get_minutes(x)<_0330pmto0430pm[1])
weekday_0430pmto0600pm = weekday.\
                        filter(lambda x: get_minutes(x)>=_0430pmto0600pm[0] and get_minutes(x)<_0430pmto0600pm[1])
weekday_0600pmto0900pm = weekday.\
                        filter(lambda x: get_minutes(x)>=_0600pmto0900pm[0] and get_minutes(x)<_0600pmto0900pm[1])
weekday_0900pmto1200am = weekday.\
                        filter(lambda x: get_minutes(x)>=_0900pmto1200am[0] and get_minutes(x)<_0900pmto1200am[1])
    
sunday_1200amto0200am = sunday.\
                        filter(lambda x: get_minutes(x)>=_1200amto0200am[0] and get_minutes(x)<_1200amto0200am[1])
sunday_0200amto0200pm = sunday.\
                        filter(lambda x: get_minutes(x)>=_0200amto0200pm[0] and get_minutes(x)<_0200amto0200pm[1])
sunday_0200pmto0400pm = sunday.\
                        filter(lambda x: get_minutes(x)>=_0200pmto0400pm[0] and get_minutes(x)<_0200pmto0400pm[1])
sunday_0400pmto1200am = sunday.\
                        filter(lambda x: get_minutes(x)>=_0400pmto1200am[0] and get_minutes(x)<_0400pmto1200am[1])

In [38]:
weekday_1200amto0600am = weekday_1200amto0600am.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(weekday_1200amto0600am.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
weekday_0600amto1000am = weekday_0600amto1000am.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(weekday_0600amto1000am.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
weekday_1000amto0330pm = weekday_1000amto0330pm.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(weekday_1000amto0330pm.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
weekday_0330pmto0430pm = weekday_0330pmto0430pm.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(weekday_0330pmto0430pm.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
weekday_0430pmto0600pm = weekday_0430pmto0600pm.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(weekday_0430pmto0600pm.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
weekday_0600pmto0900pm = weekday_0600pmto0900pm.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(weekday_0600pmto0900pm.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
weekday_0900pmto1200am = weekday_0900pmto1200am.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(weekday_0900pmto1200am.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)

sunday_1200amto0200am = sunday_1200amto0200am.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(sunday_1200amto0200am.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
sunday_0200amto0200pm = sunday_0200amto0200pm.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(sunday_0200amto0200pm.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
sunday_0200pmto0400pm = sunday_0200pmto0400pm.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(sunday_0200pmto0400pm.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)
sunday_0400pmto1200am = sunday_0400pmto1200am.map(lambda x: (x.split(',')[1][3:], 1)).\
                        union(sunday_0400pmto1200am.map(lambda x: (x.split(',')[2][:-2], 1))).\
                        reduceByKey(lambda x,y : x+y)

In [39]:
weekday_1200amto0600am.saveAsTextFile('weekday_1200amto0600am')
weekday_0600amto1000am.saveAsTextFile('weekday_0600amto1000am')
weekday_1000amto0330pm.saveAsTextFile('weekday_1000amto0330pm')
weekday_0330pmto0430pm.saveAsTextFile('weekday_0330pmto0430pm')
weekday_0430pmto0600pm.saveAsTextFile('weekday_0430pmto0600pm')
weekday_0600pmto0900pm.saveAsTextFile('weekday_0600pmto0900pm')
weekday_0900pmto1200am.saveAsTextFile('weekday_0900pmto1200am')
sunday_1200amto0200am.saveAsTextFile('sunday_1200amto0200am')
sunday_0200amto0200pm.saveAsTextFile('sunday_0200amto0200pm')
sunday_0200pmto0400pm.saveAsTextFile('sunday_0200pmto0400pm')
sunday_0400pmto1200am.saveAsTextFile('sunday_0400pmto1200am')

In [40]:
# Merging
with open('weekday_1200amto0600am.csv', 'wb') as outpt:
    for name in glob.glob('weekday_1200amto0600am/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('weekday_0600amto1000am.csv', 'wb') as outpt:
    for name in glob.glob('weekday_0600amto1000am/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('weekday_1000amto0330pm.csv', 'wb') as outpt:
    for name in glob.glob('weekday_1000amto0330pm/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('weekday_0330pmto0430pm.csv', 'wb') as outpt:
    for name in glob.glob('weekday_0330pmto0430pm/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('weekday_0430pmto0600pm.csv', 'wb') as outpt:
    for name in glob.glob('weekday_0430pmto0600pm/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('weekday_0600pmto0900pm.csv', 'wb') as outpt:
    for name in glob.glob('weekday_0600pmto0900pm/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('weekday_0900pmto1200am.csv', 'wb') as outpt:
    for name in glob.glob('weekday_0900pmto1200am/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('sunday_1200amto0200am.csv', 'wb') as outpt:
    for name in glob.glob('sunday_1200amto0200am/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('sunday_0200amto0200pm.csv', 'wb') as outpt:
    for name in glob.glob('sunday_0200amto0200pm/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('sunday_0200pmto0400pm.csv', 'wb') as outpt:
    for name in glob.glob('sunday_0200pmto0400pm/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)
            
with open('sunday_0400pmto1200am.csv', 'wb') as outpt:
    for name in glob.glob('sunday_0400pmto1200am/p*'):
        with open(name, 'rb') as inpt:
            shutil.copyfileobj(inpt, outpt)